In [1]:
from bs4 import BeautifulSoup
import requests

from IPython.core.display import display, HTML

import pandas as pd

import re

import numpy as np

import datetime

In [2]:
#Retrieve breakdown of games from october to end of march

urlList = [
    'https://www.basketball-reference.com/leagues/NBA_2017_games-october.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-november.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-december.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-january.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-february.html',
    'https://www.basketball-reference.com/leagues/NBA_2017_games-march.html'
]
soupList = []
for url in urlList:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    soupList.append(soup)
[soup1,soup2,soup3,soup4,soup5,soup6] = soupList

In [3]:
headerData = soup1.find(class_ = 'overthrow table_container').find_all('tr')[0].find_all('th')

In [4]:
column_headers = []

#get headers
for header in headerData:
    column_headers.append(header['data-stat'])
column_headers

['date_game',
 'game_start_time',
 'visitor_team_name',
 'visitor_pts',
 'home_team_name',
 'home_pts',
 'box_score_text',
 'overtimes',
 'attendance',
 'game_remarks']

In [5]:
fullDataList = []

for soup in soupList:
    cellData = soup.find(class_ = 'overthrow table_container').find_all('tr')
    for row in range(1,len(cellData)):
        rowList = []
        for cell in cellData[row].find_all(lambda tag: tag.name == 'td' or tag.name == 'th'):
            if cell.contents == []:
                value = ''
            elif cell.contents[0].name == 'a':
                if cell.contents[0].contents[0] == 'Box Score':
                    value = 'https://www.basketball-reference.com'+ cell.contents[0]['href']
                else:
                    value = cell.contents[0].contents[0]
            else:
                value = cell.contents[0]
            rowList.append(value)
        fullDataList.append(rowList)

cleanCellData = [x for x in fullDataList if x != []]

cleanCellData

[['Tue, Oct 25, 2016',
  '7:30p',
  'New York Knicks',
  '88',
  'Cleveland Cavaliers',
  '117',
  'https://www.basketball-reference.com/boxscores/201610250CLE.html',
  '',
  '20,562',
  ''],
 ['Tue, Oct 25, 2016',
  '10:00p',
  'Utah Jazz',
  '104',
  'Portland Trail Blazers',
  '113',
  'https://www.basketball-reference.com/boxscores/201610250POR.html',
  '',
  '19,446',
  ''],
 ['Tue, Oct 25, 2016',
  '10:30p',
  'San Antonio Spurs',
  '129',
  'Golden State Warriors',
  '100',
  'https://www.basketball-reference.com/boxscores/201610250GSW.html',
  '',
  '19,596',
  ''],
 ['Wed, Oct 26, 2016',
  '7:00p',
  'Dallas Mavericks',
  '121',
  'Indiana Pacers',
  '130',
  'https://www.basketball-reference.com/boxscores/201610260IND.html',
  'OT',
  '17,923',
  ''],
 ['Wed, Oct 26, 2016',
  '7:00p',
  'Miami Heat',
  '108',
  'Orlando Magic',
  '96',
  'https://www.basketball-reference.com/boxscores/201610260ORL.html',
  '',
  '19,298',
  ''],
 ['Wed, Oct 26, 2016',
  '7:30p',
  'Brooklyn N

In [6]:
#Combine Header and Data to create intiial DF

df = pd.DataFrame(cleanCellData,columns = column_headers)

In [1]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

df['Date'] = pd.to_datetime(df.apply(date_change,axis = 1))

NameError: name 'pd' is not defined

In [8]:
nbaDict = {
'Atlanta Hawks': 'ATL',
'Brooklyn Nets': 'BRK',
'Boston Celtics': 'BOS',
'Charlotte Hornets': 'CHO',
'Chicago Bulls': 'CHI',
'Cleveland Cavaliers': 'CLE',
'Dallas Mavericks': 'DAL',
'Denver Nuggets': 'DEN',
'Detroit Pistons': 'DET',
'Golden State Warriors': 'GSW',
'Houston Rockets': 'HOU',
'Indiana Pacers': 'IND',
'Los Angeles Clippers': 'LAC',
'Los Angeles Lakers': 'LAL',
'Memphis Grizzlies': 'MEM',
'Miami Heat': 'MIA',
'Milwaukee Bucks': 'MIL',
'Minnesota Timberwolves': 'MIN',
'New Orleans Pelicans': 'NOP',
'New York Knicks': 'NYK',
'Oklahoma City Thunder': 'OKC',
'Orlando Magic': 'ORL',
'Philadelphia 76ers': 'PHI',
'Phoenix Suns': 'PHO',
'Portland Trail Blazers': 'POR',
'Sacramento Kings': 'SAC',
'San Antonio Spurs': 'SAS',
'Toronto Raptors': 'TOR',
'Utah Jazz': 'UTA',
'Washington Wizards': 'WAS'}

In [9]:
statColumns = ['vis FG','vis FGA','vis FG %','vis 3P','vis 3PA',
    'vis 3P%','vis FT','vis FTA','vis FT%','vis ORB',
    'vis DRB','vis TRB','vis AST','vis STL','vis BLK',
    'vis TOV','vis PF','home FG','home FGA','home FG %',
    'home 3P','home 3PA','home 3P%','home FT','home FTA',
    'home FT%','home ORB','home DRB','home TRB','home AST',
    'home STL','home BLK','home TOV','home PF']

In [10]:
def statRecorder(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [row['visitor_team_name'],row['home_team_name']]
    statsList = []
    for team in team_list:
        nbaTeam = nbaDict[team]
        boxScoreLink = "box-{}-game-basic".format(nbaTeam)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for cell in totalStats[1:-2]:
            statsList.append(cell.contents[0])
    return(statsList)

In [11]:
#Create new stats list incrementally and then concatenate after
stats_list = []

In [12]:
zeroBatch = df[0:1].apply(statRecorder,axis = 1)
for i in zeroBatch:
    stats_list.insert(0,i)

In [13]:
firstBatch = df[1:101].apply(statRecorder,axis = 1)
for i in firstBatch:
    stats_list.append(i)

In [14]:
secondBatch = df[101:201].apply(statRecorder,axis = 1)
for i in secondBatch:
    stats_list.append(i)

In [15]:
thirdBatch = df[201:301].apply(statRecorder,axis = 1)
for i in thirdBatch:
    stats_list.append(i)

In [16]:
fourthBatch = df[301:401].apply(statRecorder,axis = 1)
for i in fourthBatch:
    stats_list.append(i)

In [17]:
fifthBatch = df[401:501].apply(statRecorder,axis = 1)
for i in fifthBatch:
    stats_list.append(i)

In [18]:
sixthBatch = df[501:548].apply(statRecorder,axis = 1)
for i in sixthBatch:
    stats_list.append(i)

In [19]:
df[545:548]

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,Date
545,"Fri, Jan 6, 2017",7:30p,Cleveland Cavaliers,116,Brooklyn Nets,108,https://www.basketball-reference.com/boxscores...,,"17,732",,2017-01-06
546,"Fri, Jan 6, 2017",8:00p,New York Knicks,116,Milwaukee Bucks,111,https://www.basketball-reference.com/boxscores...,,"18,717",,2017-01-06
547,"Fri, Jan 6, 2017",10:30p,Memphis Grizzlies,128,Golden State Warriors,119,https://www.basketball-reference.com/boxscores...,OT,"19,596",,2017-01-06


In [20]:
nbaDictLower = {
'Atlanta Hawks': 'atl',
'Brooklyn Nets': 'brk',
'Boston Celtics': 'bos',
'Charlotte Hornets': 'cho',
'Chicago Bulls': 'chi',
'Cleveland Cavaliers': 'cle',
'Dallas Mavericks': 'dal',
'Denver Nuggets': 'den',
'Detroit Pistons': 'det',
'Golden State Warriors': 'gsw',
'Houston Rockets': 'hou',
'Indiana Pacers': 'ind',
'Los Angeles Clippers': 'lac',
'Los Angeles Lakers': 'lal',
'Memphis Grizzlies': 'mem',
'Miami Heat': 'mia',
'Milwaukee Bucks': 'mil',
'Minnesota Timberwolves': 'min',
'New Orleans Pelicans': 'nop',
'New York Knicks': 'nyk',
'Oklahoma City Thunder': 'okc',
'Orlando Magic': 'orl',
'Philadelphia 76ers': 'phi',
'Phoenix Suns': 'pho',
'Portland Trail Blazers': 'por',
'Sacramento Kings': 'sac',
'San Antonio Spurs': 'sas',
'Toronto Raptors': 'tor',
'Utah Jazz': 'uta',
'Washington Wizards': 'was'}

In [21]:
def statRecorder2(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [row['visitor_team_name'],row['home_team_name']]
    statsList = []
    for team in team_list:
        nbaTeam = nbaDictLower[team]
        boxScoreLink = "box_{}_basic".format(nbaTeam)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for cell in totalStats[1:-2]:
            statsList.append(cell.contents[0])
    return(statsList)

In [22]:
seventhBatch = df[548:701].apply(statRecorder,axis = 1)
for i in seventhBatch:
    stats_list.append(i)

In [23]:
eighthBatch = df[701:801].apply(statRecorder,axis = 1)
for i in eighthBatch:
    stats_list.append(i)

In [24]:
ninthBatch = df[801:901].apply(statRecorder,axis = 1)
for i in ninthBatch:
    stats_list.append(i)

In [25]:
tenthBatch = df[901:1001].apply(statRecorder,axis = 1)
for i in tenthBatch:
    stats_list.append(i)

In [26]:
eleventhBatch = df[1001:1152].apply(statRecorder,axis = 1)
for i in eleventhBatch:
    stats_list.append(i)

In [27]:
#Create new pd with stats list and stats columns
statsdf = pd.DataFrame(stats_list,columns = statColumns)
statsdf.head()

,vis FG,vis FGA,vis FG %,vis 3P,vis 3PA,vis 3P%,vis FT,vis FTA,vis FT%,vis ORB,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,32,87,.368,9,27,.333,15,20,.750,13,...,19,.737,11,40,51,31,12,5,14,22
1,40,82,.488,8,24,.333,16,16,1.000,6,...,22,1.000,5,29,34,22,5,3,12,18
2,47,98,.480,12,24,.500,23,26,.885,21,...,18,.722,8,27,35,24,11,6,16,19
3,45,104,.433,18,48,.375,13,18,.722,10,...,34,.765,8,44,52,30,11,6,16,23
4,47,97,.485,4,16,.250,10,16,.625,16,...,28,.786,15,30,45,20,5,4,11,15


In [28]:
#concatenate original df with stats df
resultdf = pd.concat([df, statsdf], axis=1)
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,"Tue, Oct 25, 2016",7:30p,New York Knicks,88,Cleveland Cavaliers,117,https://www.basketball-reference.com/boxscores...,,"20,562",,...,19,.737,11,40,51,31,12,5,14,22
1,"Tue, Oct 25, 2016",10:00p,Utah Jazz,104,Portland Trail Blazers,113,https://www.basketball-reference.com/boxscores...,,"19,446",,...,22,1.000,5,29,34,22,5,3,12,18
2,"Tue, Oct 25, 2016",10:30p,San Antonio Spurs,129,Golden State Warriors,100,https://www.basketball-reference.com/boxscores...,,"19,596",,...,18,.722,8,27,35,24,11,6,16,19
3,"Wed, Oct 26, 2016",7:00p,Dallas Mavericks,121,Indiana Pacers,130,https://www.basketball-reference.com/boxscores...,OT,"17,923",,...,34,.765,8,44,52,30,11,6,16,23
4,"Wed, Oct 26, 2016",7:00p,Miami Heat,108,Orlando Magic,96,https://www.basketball-reference.com/boxscores...,,"19,298",,...,28,.786,15,30,45,20,5,4,11,15


In [29]:
#Output to csv for saving purposes
export_csv = resultdf.to_csv (r'C:\Users\jeromerufin\Desktop\Metis\stats_data.csv') #Don't forget to add '.csv' at the end of the path

In [30]:
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,"Tue, Oct 25, 2016",7:30p,New York Knicks,88,Cleveland Cavaliers,117,https://www.basketball-reference.com/boxscores...,,"20,562",,...,19,.737,11,40,51,31,12,5,14,22
1,"Tue, Oct 25, 2016",10:00p,Utah Jazz,104,Portland Trail Blazers,113,https://www.basketball-reference.com/boxscores...,,"19,446",,...,22,1.000,5,29,34,22,5,3,12,18
2,"Tue, Oct 25, 2016",10:30p,San Antonio Spurs,129,Golden State Warriors,100,https://www.basketball-reference.com/boxscores...,,"19,596",,...,18,.722,8,27,35,24,11,6,16,19
3,"Wed, Oct 26, 2016",7:00p,Dallas Mavericks,121,Indiana Pacers,130,https://www.basketball-reference.com/boxscores...,OT,"17,923",,...,34,.765,8,44,52,30,11,6,16,23
4,"Wed, Oct 26, 2016",7:00p,Miami Heat,108,Orlando Magic,96,https://www.basketball-reference.com/boxscores...,,"19,298",,...,28,.786,15,30,45,20,5,4,11,15


In [31]:
def win_percent(row):
    testurl = row['box_score_text']
    testresponse = requests.get(testurl)
    testpage = testresponse.text
    testsoup = BeautifulSoup(testpage,"lxml")
    wins_team = int(testsoup.findAll('div', text = re.compile("-"))[0].contents[0][0:].split('-')[0])
    losses_team = int(testsoup.findAll('div', text = re.compile("-"))[0].contents[0][0:].split('-')[1])
    win_pct_team = wins_team/(wins_team+losses_team)
    wins_opp = int(testsoup.findAll('div', text = re.compile("-"))[1].contents[0][0:].split('-')[0])
    losses_opp = int(testsoup.findAll('div', text = re.compile("-"))[1].contents[0][0:].split('-')[1])
    win_pct_opp = wins_opp/(wins_opp+losses_opp)
    win_list = [win_pct_team, win_pct_opp]
    print(win_list)
    return(win_list)



In [32]:
win_first_batch = resultdf[0:101]
win_second_batch = resultdf[101:201]
win_third_batch = resultdf[201:301]
win_fourth_batch = resultdf[301:401]
win_fifth_batch = resultdf[401:501]
win_sixth_batch = resultdf[501:601]
win_seventh_batch = resultdf[601:701]
win_eighth_batch = resultdf[701:801]
win_ninth_batch = resultdf[801:901]
win_tenth_batch = resultdf[901:1001]
win_eleventh_batch = resultdf[1001:1152]

In [33]:
win_final_list = []
for i in win_first_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.0, 1.0]
[0.0, 1.0]
[1.0, 0.0]
[0.0, 1.0]
[1.0, 0.0]
[0.0, 1.0]
[0.0, 1.0]
[1.0, 0.0]
[1.0, 0.0]
[0.0, 1.0]
[1.0, 0.0]
[1.0, 0.0]
[0.0, 1.0]
[0.0, 1.0]
[0.5, 1.0]
[1.0, 0.5]
[1.0, 0.5]
[1.0, 0.5]
[0.5, 0.5]
[0.0, 0.5]
[0.0, 1.0]
[1.0, 0.5]
[0.5, 0.0]
[0.5, 0.5]
[0.5, 0.0]
[1.0, 0.0]
[0.6666666666666666, 0.6666666666666666]
[0.0, 1.0]
[0.5, 0.5]
[0.0, 1.0]
[0.3333333333333333, 0.5]
[0.3333333333333333, 1.0]
[0.6666666666666666, 0.5]
[0.0, 0.6666666666666666]
[0.3333333333333333, 1.0]
[0.3333333333333333, 0.6666666666666666]
[0.6666666666666666, 0.0]
[1.0, 0.3333333333333333]
[0.3333333333333333, 1.0]
[0.0, 0.6666666666666666]
[0.0, 0.6666666666666666]
[0.5, 1.0]
[1.0, 0.25]
[0.3333333333333333, 0.6666666666666666]
[0.0, 1.0]
[0.5, 1.0]
[0.25, 0.5]
[0.25, 0.0]
[0.3333333333333333, 0.75]
[0.4, 0.5]
[0.5, 0.3333333333333333]
[0.5, 0.0]
[0.5, 0.8]
[0.75, 0.5]
[0.0, 0.75]
[0.75, 0.0]
[0.4, 0.75]
[0.6, 0.4]
[0.6, 0.25]
[0.75, 0.75]
[0.0, 0.6]
[0.0, 0.6]
[0.4, 0.2]
[1.0, 0.75]
[0.33333333333

In [34]:
for i in win_second_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.42857142857142855, 0.5]
[0.25, 0.625]
[0.2857142857142857, 0.5]
[0.0, 0.4444444444444444]
[0.5555555555555556, 0.8571428571428571]
[0.0, 0.5]
[0.375, 0.42857142857142855]
[0.2857142857142857, 0.375]
[0.42857142857142855, 0.2857142857142857]
[0.5, 0.75]
[0.7142857142857143, 0.75]
[0.5, 0.3333333333333333]
[0.625, 0.625]
[0.25, 0.75]
[0.5555555555555556, 0.875]
[0.5555555555555556, 0.2857142857142857]
[0.1111111111111111, 0.5]
[0.7777777777777778, 0.375]
[0.5555555555555556, 0.4]
[0.75, 0.75]
[0.6, 0.3333333333333333]
[0.4444444444444444, 0.125]
[0.875, 0.25]
[0.375, 0.5]
[0.8888888888888888, 0.6666666666666666]
[0.4444444444444444, 0.6666666666666666]
[0.36363636363636365, 0.6]
[0.5555555555555556, 0.4]
[0.6, 0.1]
[0.1111111111111111, 0.7777777777777778]
[0.3333333333333333, 0.7777777777777778]
[0.2222222222222222, 0.6]
[0.7, 0.5555555555555556]
[0.6363636363636364, 0.25]
[0.9, 0.25]
[0.4444444444444444, 0.5555555555555556]
[0.5, 0.3333333333333333]
[0.4444444444444444, 0.3]
[0.66666

In [35]:
for i in win_third_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.26666666666666666, 0.3076923076923077]
[0.6428571428571429, 0.4]
[0.42857142857142855, 0.46153846153846156]
[0.5714285714285714, 0.3076923076923077]
[0.15384615384615385, 0.7857142857142857]
[0.5714285714285714, 0.8666666666666667]
[0.3333333333333333, 0.6428571428571429]
[0.5, 0.5]
[0.6, 0.42857142857142855]
[0.5333333333333333, 0.5333333333333333]
[0.8, 0.5714285714285714]
[0.47058823529411764, 0.8461538461538461]
[0.6666666666666666, 0.4375]
[0.3125, 0.4]
[0.6666666666666666, 0.26666666666666666]
[0.6, 0.2857142857142857]
[0.2857142857142857, 0.4375]
[0.6, 0.6]
[0.875, 0.14285714285714285]
[0.4, 0.5]
[0.2857142857142857, 0.375]
[0.5, 0.8666666666666667]
[0.5, 0.4]
[0.8125, 0.5625]
[0.35714285714285715, 0.375]
[0.13333333333333333, 0.8571428571428571]
[0.8235294117647058, 0.47058823529411764]
[0.5333333333333333, 0.5333333333333333]
[0.625, 0.25]
[0.26666666666666666, 0.47058823529411764]
[0.3333333333333333, 0.625]
[0.625, 0.42857142857142855]
[0.625, 0.5294117647058824]
[0.52941

In [36]:
for i in win_fourth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.5, 0.7272727272727273]
[0.38095238095238093, 0.19047619047619047]
[0.6363636363636364, 0.45454545454545453]
[0.3684210526315789, 0.25]
[0.7368421052631579, 0.6666666666666666]
[0.5454545454545454, 0.55]
[0.5714285714285714, 0.6666666666666666]
[0.8095238095238095, 0.5263157894736842]
[0.6363636363636364, 0.3181818181818182]
[0.5714285714285714, 0.2]
[0.47619047619047616, 0.8571428571428571]
[0.5909090909090909, 0.43478260869565216]
[0.45454545454545453, 0.35]
[0.5238095238095238, 0.5217391304347826]
[0.5714285714285714, 0.3333333333333333]
[0.18181818181818182, 0.6521739130434783]
[0.8181818181818182, 0.2857142857142857]
[0.2857142857142857, 0.6086956521739131]
[0.5, 0.5909090909090909]
[0.5909090909090909, 0.43478260869565216]
[0.3181818181818182, 0.4782608695652174]
[0.36363636363636365, 0.2857142857142857]
[0.4166666666666667, 0.6818181818181818]
[0.5217391304347826, 0.55]
[0.75, 0.5454545454545454]
[0.38095238095238093, 0.19047619047619047]
[0.5, 0.2727272727272727]
[0.863636363

In [37]:
for i in win_fifth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.75, 0.2692307692307692]
[0.5185185185185185, 0.4074074074074074]
[0.4482758620689655, 0.8571428571428571]
[0.7142857142857143, 0.46153846153846156]
[0.37037037037037035, 0.25925925925925924]
[0.6428571428571429, 0.6206896551724138]
[0.5555555555555556, 0.32142857142857145]
[0.7037037037037037, 0.41379310344827586]
[0.2692307692307692, 0.25925925925925924]
[0.3103448275862069, 0.8148148148148148]
[0.4444444444444444, 0.5172413793103449]
[0.4666666666666667, 0.5185185185185185]
[0.2857142857142857, 0.2962962962962963]
[0.5, 0.5714285714285714]
[0.25, 0.42857142857142855]
[0.3548387096774194, 0.5517241379310345]
[0.3333333333333333, 0.25]
[0.43333333333333335, 0.3103448275862069]
[0.5, 0.5357142857142857]
[0.25925925925925924, 0.7142857142857143]
[0.8214285714285714, 0.7241379310344828]
[0.5714285714285714, 0.6]
[0.7692307692307693, 0.5]
[0.6206896551724138, 0.8620689655172413]
[0.41379310344827586, 0.7241379310344828]
[0.43333333333333335, 0.39285714285714285]
[0.48148148148148145, 0.

In [38]:
for i in win_sixth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.5, 0.47058823529411764]
[0.78125, 0.5588235294117647]
[0.48484848484848486, 0.7428571428571429]
[0.6111111111111112, 0.6176470588235294]
[0.29411764705882354, 0.6176470588235294]
[0.7941176470588235, 0.5294117647058824]
[0.42857142857142855, 0.4857142857142857]
[0.4444444444444444, 0.2857142857142857]
[0.4166666666666667, 0.3235294117647059]
[0.696969696969697, 0.32432432432432434]
[0.3888888888888889, 0.7878787878787878]
[0.6, 0.5151515151515151]
[0.6285714285714286, 0.24242424242424243]
[0.4444444444444444, 0.47058823529411764]
[0.5428571428571428, 0.4857142857142857]
[0.48484848484848486, 0.75]
[0.4117647058823529, 0.8571428571428571]
[0.2857142857142857, 0.6216216216216216]
[0.3142857142857143, 0.2727272727272727]
[0.6111111111111112, 0.6]
[0.5, 0.43243243243243246]
[0.47058823529411764, 0.3142857142857143]
[0.6764705882352942, 0.8]
[0.4411764705882353, 0.4]
[0.2777777777777778, 0.3055555555555556]
[0.5945945945945946, 0.34210526315789475]
[0.5833333333333334, 0.5555555555555556

In [39]:
for i in win_seventh_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.775, 0.325]
[0.3157894736842105, 0.5128205128205128]
[0.40476190476190477, 0.6190476190476191]
[0.34146341463414637, 0.325]
[0.5128205128205128, 0.575]
[0.43902439024390244, 0.675]
[0.7441860465116279, 0.2]
[0.5, 0.5813953488372093]
[0.5952380952380952, 0.4]
[0.4418604651162791, 0.3333333333333333]
[0.5853658536585366, 0.42857142857142855]
[0.4186046511627907, 0.525]
[0.3333333333333333, 0.5]
[0.38095238095238093, 0.525]
[0.3953488372093023, 0.41025641025641024]
[0.4878048780487805, 0.6341463414634146]
[0.725, 0.8536585365853658]
[0.627906976744186, 0.3170731707317073]
[0.5813953488372093, 0.6744186046511628]
[0.6829268292682927, 0.1951219512195122]
[0.7272727272727273, 0.2857142857142857]
[0.34146341463414637, 0.4883720930232558]
[0.3333333333333333, 0.7804878048780488]
[0.425, 0.32608695652173914]
[0.4090909090909091, 0.5]
[0.6666666666666666, 0.35]
[0.5681818181818182, 0.5365853658536586]
[0.4418604651162791, 0.6190476190476191]
[0.5714285714285714, 0.45454545454545453]
[0.487804

In [40]:
for i in win_eighth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.5625, 0.4375]
[0.40425531914893614, 0.4791666666666667]
[0.44680851063829785, 0.375]
[0.6170212765957447, 0.44680851063829785]
[0.625, 0.851063829787234]
[0.19148936170212766, 0.3829787234042553]
[0.45652173913043476, 0.3191489361702128]
[0.5714285714285714, 0.6122448979591837]
[0.42857142857142855, 0.5833333333333334]
[0.5833333333333334, 0.6956521739130435]
[0.6862745098039216, 0.5319148936170213]
[0.574468085106383, 0.3958333333333333]
[0.3877551020408163, 0.6041666666666666]
[0.3695652173913043, 0.4897959183673469]
[0.3617021276595745, 0.7659574468085106]
[0.8541666666666666, 0.42857142857142855]
[0.3958333333333333, 0.3829787234042553]
[0.1875, 0.3877551020408163]
[0.4375, 0.625]
[0.38, 0.3958333333333333]
[0.6808510638297872, 0.375]
[0.58, 0.3125]
[0.3877551020408163, 0.6122448979591837]
[0.42, 0.5833333333333334]
[0.3877551020408163, 0.6923076923076923]
[0.5714285714285714, 0.7708333333333334]
[0.46938775510204084, 0.44]
[0.44680851063829785, 0.3333333333333333]
[0.3877551020

In [41]:
for i in win_ninth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.8490566037735849, 0.5892857142857143]
[0.3392857142857143, 0.4230769230769231]
[0.3888888888888889, 0.37037037037037035]
[0.5471698113207547, 0.6037735849056604]
[0.48148148148148145, 0.3148148148148148]
[0.5740740740740741, 0.4074074074074074]
[0.6111111111111112, 0.4444444444444444]
[0.4339622641509434, 0.5370370370370371]
[0.4444444444444444, 0.6981132075471698]
[0.43636363636363634, 0.37037037037037035]
[0.8518518518518519, 0.5636363636363636]
[0.35714285714285715, 0.4074074074074074]
[0.3090909090909091, 0.7017543859649122]
[0.6481481481481481, 0.6181818181818182]
[0.4727272727272727, 0.38181818181818183]
[0.7592592592592593, 0.4107142857142857]
[0.4727272727272727, 0.5818181818181818]
[0.38181818181818183, 0.41818181818181815]
[0.38181818181818183, 0.43636363636363634]
[0.7636363636363637, 0.5272727272727272]
[0.5964912280701754, 0.16363636363636364]
[0.3684210526315789, 0.42857142857142855]
[0.4642857142857143, 0.4444444444444444]
[0.5535714285714286, 0.6111111111111112]
[0.6

In [42]:
for i in win_tenth_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.5081967213114754, 0.7796610169491526]
[0.4098360655737705, 0.6065573770491803]
[0.6935483870967742, 0.6]
[0.1694915254237288, 0.4098360655737705]
[0.819672131147541, 0.5081967213114754]
[0.4262295081967213, 0.3114754098360656]
[0.5737704918032787, 0.4166666666666667]
[0.7, 0.5573770491803278]
[0.45161290322580644, 0.3709677419354839]
[0.4032258064516129, 0.3770491803278688]
[0.5967741935483871, 0.6]
[0.5901639344262295, 0.45]
[0.5806451612903226, 0.4098360655737705]
[0.5645161290322581, 0.3225806451612903]
[0.16666666666666666, 0.6129032258064516]
[0.7833333333333333, 0.3870967741935484]
[0.6451612903225806, 0.3064516129032258]
[0.4838709677419355, 0.3709677419354839]
[0.6885245901639344, 0.4603174603174603]
[0.5873015873015873, 0.45901639344262296]
[0.5967741935483871, 0.5]
[0.43548387096774194, 0.45161290322580644]
[0.5714285714285714, 0.6984126984126984]
[0.4032258064516129, 0.7868852459016393]
[0.16393442622950818, 0.4262295081967213]
[0.5161290322580645, 0.5483870967741935]
[0.

In [43]:
for i in win_eleventh_batch.apply(win_percent,axis=1):
    win_final_list.append(i)

[0.3582089552238806, 0.7910447761194029]
[0.4264705882352941, 0.5147058823529411]
[0.43283582089552236, 0.6119402985074627]
[0.417910447761194, 0.6323529411764706]
[0.6323529411764706, 0.4852941176470588]
[0.39705882352941174, 0.4852941176470588]
[0.5588235294117647, 0.47058823529411764]
[0.29411764705882354, 0.6911764705882353]
[0.44776119402985076, 0.7761194029850746]
[0.39705882352941174, 0.3235294117647059]
[0.4925373134328358, 0.5882352941176471]
[0.6231884057971014, 0.6716417910447762]
[0.5735294117647058, 0.5735294117647058]
[0.5652173913043478, 0.5441176470588235]
[0.19402985074626866, 0.391304347826087]
[0.5797101449275363, 0.4852941176470588]
[0.34782608695652173, 0.7941176470588235]
[0.4264705882352941, 0.36764705882352944]
[0.463768115942029, 0.6176470588235294]
[0.6376811594202898, 0.19117647058823528]
[0.5797101449275363, 0.4782608695652174]
[0.4117647058823529, 0.4927536231884058]
[0.6811594202898551, 0.4057971014492754]
[0.35714285714285715, 0.3188405797101449]
[0.5, 0.

In [44]:
win_columns = ['Team_Win_Pct','Opp_Win_Pct']

win_pct_df = pd.DataFrame(win_final_list,columns = win_columns)

In [45]:
resultdf = pd.concat([resultdf, win_pct_df], axis=1)

In [47]:
resultdf

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,...,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF,Team_Win_Pct,Opp_Win_Pct
0,"Tue, Oct 25, 2016",7:30p,New York Knicks,88,Cleveland Cavaliers,117,https://www.basketball-reference.com/boxscores...,,"20,562",,...,11,40,51,31,12,5,14,22,0.000000,1.000000
1,"Tue, Oct 25, 2016",10:00p,Utah Jazz,104,Portland Trail Blazers,113,https://www.basketball-reference.com/boxscores...,,"19,446",,...,5,29,34,22,5,3,12,18,0.000000,1.000000
2,"Tue, Oct 25, 2016",10:30p,San Antonio Spurs,129,Golden State Warriors,100,https://www.basketball-reference.com/boxscores...,,"19,596",,...,8,27,35,24,11,6,16,19,1.000000,0.000000
3,"Wed, Oct 26, 2016",7:00p,Dallas Mavericks,121,Indiana Pacers,130,https://www.basketball-reference.com/boxscores...,OT,"17,923",,...,8,44,52,30,11,6,16,23,0.000000,1.000000
4,"Wed, Oct 26, 2016",7:00p,Miami Heat,108,Orlando Magic,96,https://www.basketball-reference.com/boxscores...,,"19,298",,...,15,30,45,20,5,4,11,15,1.000000,0.000000
5,"Wed, Oct 26, 2016",7:30p,Brooklyn Nets,117,Boston Celtics,122,https://www.basketball-reference.com/boxscores...,,"18,624",,...,12,35,47,36,13,9,19,20,0.000000,1.000000
6,"Wed, Oct 26, 2016",7:30p,Detroit Pistons,91,Toronto Raptors,109,https://www.basketball-reference.com/boxscores...,,"19,800",,...,14,37,51,17,8,0,11,22,0.000000,1.000000
7,"Wed, Oct 26, 2016",8:00p,Denver Nuggets,107,New Orleans Pelicans,102,https://www.basketball-reference.com/boxscores...,,"15,869",,...,5,28,33,31,16,5,11,29,1.000000,0.000000
8,"Wed, Oct 26, 2016",8:00p,Charlotte Hornets,107,Milwaukee Bucks,96,https://www.basketball-reference.com/boxscores...,,"18,717",,...,10,36,46,23,4,5,8,22,1.000000,0.000000
9,"Wed, Oct 26, 2016",8:00p,Minnesota Timberwolves,98,Memphis Grizzlies,102,https://www.basketball-reference.com/boxscores...,,"18,119",,...,11,34,45,19,6,11,13,25,0.000000,1.000000


In [48]:
#visitor stats
visitor_df = resultdf.iloc[:,np.r_[0:28,45]]

#home stats
home_df = resultdf.iloc[:,np.r_[0:2,4:6,2:4,6:11,28:45,46]]


In [49]:
#Create bool indicator if the team is home or away
list_of_zeros = [0]*visitor_df.shape[0]
list_of_ones = [1]*home_df.shape[0]

away_bool_df = pd.DataFrame(list_of_zeros,columns = ['Home Team'])
home_bool_df = pd.DataFrame(list_of_ones,columns = ['Home Team'])

In [50]:
#Add new column for binary home or away
visitor_df = pd.concat([visitor_df, away_bool_df], axis=1)
home_df = pd.concat([home_df, home_bool_df], axis=1)

In [51]:
visitor_df.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text', 'overtimes',
       'attendance', 'game_remarks', 'Date', 'FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF','Team_win_pct','Home Team']


home_df.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text', 'overtimes',
       'attendance', 'game_remarks', 'Date', 'FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF','Team_win_pct','Home Team']

In [52]:
#Combine dfs and clean-up unnecessary columns
frames = [visitor_df, home_df]
agg_data = pd.concat(frames,ignore_index=True)
agg_data['date_game'] = agg_data['Date']
del agg_data['Date']

In [53]:
ordered_agg_data = agg_data.sort_values(by = ['team_name','date_game']).reset_index()

In [54]:
ordered_agg_data.head()

,index,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,overtimes,attendance,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Team_win_pct,Home Team
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,"19,049",...,14,38,52,28,13,7,21,19,1.00,1
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,"16,312",...,6,41,47,31,11,7,12,14,1.00,0
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,"12,501",...,17,26,43,24,11,6,13,20,1.00,1
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,"13,800",...,10,27,37,30,10,1,18,19,0.75,1
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,"14,663",...,10,37,47,22,12,3,18,24,0.60,0


In [55]:
rolling_stat_data = agg_data.sort_values(by = ['team_name','date_game'])

testdata_grouped_rolling = rolling_stat_data.groupby('team_name')[['team_pts','opposing_team_pts','FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF']].rolling(window=30, min_periods=1).mean().reset_index()

In [56]:
del testdata_grouped_rolling['level_1']
del testdata_grouped_rolling['team_name']

In [57]:
testdata_grouped_rolling.columns = ['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF']

In [58]:
full_stats_data = pd.concat([ordered_agg_data ,testdata_grouped_rolling], axis=1)
full_stats_data.head(5)

,index,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,overtimes,attendance,...,Avg1 FTA,Avg1 FT%,Avg1 ORB,Avg1 DRB,Avg1 TRB,Avg1 AST,Avg1 STL,Avg1 BLK,Avg1 TOV,Avg1 PF
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,"19,049",...,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,"16,312",...,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,"12,501",...,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,"13,800",...,29.250000,0.710000,11.750000,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,"14,663",...,30.600000,0.695800,11.400000,33.8,45.200000,27.000000,11.400000,4.800000,16.400000,19.200000


In [59]:
#Shift average stats so each row has the average of the games previously
full_stats_data[['Avg Pts','Avg Opp Pts','Avg FG','Avg FGA','Avg FG %',
       'Avg 3P','Avg 3PA','Avg 3P%','Avg FT','Avg FTA','Avg FT%',
       'Avg ORB','Avg DRB','Avg TRB','Avg AST','Avg STL','Avg BLK',
       'Avg TOV','Avg PF']] = full_stats_data.groupby('team_name')['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF'].apply(lambda grp: grp.shift(1))

In [60]:
#Deleted non-shifted columns

full_stats_data = full_stats_data.drop(['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF'],axis =1)

In [61]:
full_stats_data['Previous Game Data'] = (full_stats_data.groupby('team_name')['date_game']
                                            .apply(lambda grp: grp.shift(1)))

In [62]:
#Do calculation for back to back games
full_stats_data['Time between games'] = full_stats_data['date_game'] - full_stats_data['Previous Game Data']

In [63]:
full_stats_data['Win_pct'] = (full_stats_data.groupby('team_name')['Team_win_pct']
                                            .apply(lambda grp: grp.shift(1)))

In [64]:
#Replicate df to join, so each line has team average and opposing team average

rep_full_stats_data = full_stats_data

rep_full_stats_data.head()

,index,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,overtimes,attendance,...,Avg DRB,Avg TRB,Avg AST,Avg STL,Avg BLK,Avg TOV,Avg PF,Previous Game Data,Time between games,Win_pct
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,"19,049",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,"16,312",...,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2016-10-27,2 days,1.00
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,"12,501",...,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2016-10-29,2 days,1.00
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,"13,800",...,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2016-10-31,2 days,1.00
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,"14,663",...,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2016-11-02,2 days,0.75


In [65]:
pd.set_option('display.max_columns', 999)

In [66]:
newDf = pd.merge(full_stats_data, rep_full_stats_data, left_on = ['date_game','opposing_team'],right_on = ['date_game','team_name'])

newDf.head()


,index_x,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_team_pts_x,box_score_text_x,overtimes_x,attendance_x,game_remarks_x,FG_x,FGA_x,FG %_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,Team_win_pct_x,Home Team_x,Avg Pts_x,Avg Opp Pts_x,Avg FG_x,Avg FGA_x,Avg FG %_x,Avg 3P_x,Avg 3PA_x,Avg 3P%_x,Avg FT_x,Avg FTA_x,Avg FT%_x,Avg ORB_x,Avg DRB_x,Avg TRB_x,Avg AST_x,Avg STL_x,Avg BLK_x,Avg TOV_x,Avg PF_x,Previous Game Data_x,Time between games_x,Win_pct_x,index_y,game_start_time_y,team_name_y,team_pts_y,opposing_team_y,opposing_team_pts_y,box_score_text_y,overtimes_y,attendance_y,game_remarks_y,FG_y,FGA_y,FG %_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,Team_win_pct_y,Home Team_y,Avg Pts_y,Avg Opp Pts_y,Avg FG_y,Avg FGA_y,Avg FG %_y,Avg 3P_y,Avg 3PA_y,Avg 3P%_y,Avg FT_y,Avg FTA_y,Avg FT%_y,Avg ORB_y,Avg DRB_y,Avg TRB_y,Avg AST_y,Avg STL_y,Avg BLK_y,Avg TOV_y,Avg PF_y,Previous Game Data_y,Time between games_y,Win_pct_y
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,"19,049",,44,88,.500,12,26,.462,14,18,.778,14,38,52,28,13,7,21,19,1.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,13,7:30p,Washington Wizards,99,Atlanta Hawks,114,https://www.basketball-reference.com/boxscores...,,"19,049",,40,94,.426,6,20,.300,13,18,.722,12,28,40,25,14,4,19,20,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,"16,312",,42,85,.494,5,20,.250,15,23,.652,6,41,47,31,11,7,12,14,1.00,0,114.0,99.000000,44.000000,88.00,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2016-10-27,2 days,1.00,1160,12:30p,Philadelphia 76ers,72,Atlanta Hawks,104,https://www.basketball-reference.com/boxscores...,,"16,312",,28,80,.350,5,18,.278,11,15,.733,4,35,39,16,9,5,20,20,0.00,1,97.000000,103.000000,36.0,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.00,10.000000,13.000000,22.000000,2016-10-26,3 days,0.000000
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,"12,501",,33,79,.418,11,30,.367,29,47,.617,17,26,43,24,11,6,13,20,1.00,1,109.0,85.500000,43.000000,86.50,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2016-10-29,2 days,1.00,41,7:30p,Sacramento Kings,95,Atlanta Hawks,106,https://www.basketball-reference.com/boxscores...,,"12,501",,36,75,.480,10,23,.435,13,19,.684,6,32,38,26,10,6,19,34,0.50,0,104.333333,99.666667,37.0,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.00,3.333333,13.666667,21.666667,2016-10-29,2 days,0.666667
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,"13,800",,41,85,.482,11,32,.344,23,29,.793,10,27,37,30,10,1,18,19,0.75,1,108.0,88.666667,39.666667,84.00,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2016-10-31,2 days,1.00,56,7:30p,Los Angeles Lakers,123,Atlanta Hawks,116,https://www.basketball-reference.com/boxscores...,,"13,800",,48,90,.533,9,22,.409,18,20,.900,11,31,42,24,11,9,15,25,0.40,0,103.250000,109.500000,38.0,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.75,3.750000,17.250000,24.000000,2016-11-01,1 days,0.250000
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,"14,663",,32,83,.386,5,25,.200,23,36,.639,10,37,

In [67]:
cleandNewDf2 = newDf.drop(newDf.loc[:,'index_y':'game_remarks_y'],axis = 1)

In [ ]:
# cleandNewDf3 = cleandNewDf2.drop(cleandNewDf2.loc[:,'Avg1 Pts_y':'Avg1 PF_y'],axis = 1)

In [ ]:
# cleandNewDf3.head()

In [68]:
cleandNewDf4 = cleandNewDf2.drop(cleandNewDf2.loc[:,'FG_x':'PF_x'],axis = 1)

In [69]:
cleandNewDf4.head()

,index_x,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_team_pts_x,box_score_text_x,overtimes_x,attendance_x,game_remarks_x,Team_win_pct_x,Home Team_x,Avg Pts_x,Avg Opp Pts_x,Avg FG_x,Avg FGA_x,Avg FG %_x,Avg 3P_x,Avg 3PA_x,Avg 3P%_x,Avg FT_x,Avg FTA_x,Avg FT%_x,Avg ORB_x,Avg DRB_x,Avg TRB_x,Avg AST_x,Avg STL_x,Avg BLK_x,Avg TOV_x,Avg PF_x,Previous Game Data_x,Time between games_x,Win_pct_x,FG_y,FGA_y,FG %_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,Team_win_pct_y,Home Team_y,Avg Pts_y,Avg Opp Pts_y,Avg FG_y,Avg FGA_y,Avg FG %_y,Avg 3P_y,Avg 3PA_y,Avg 3P%_y,Avg FT_y,Avg FTA_y,Avg FT%_y,Avg ORB_y,Avg DRB_y,Avg TRB_y,Avg AST_y,Avg STL_y,Avg BLK_y,Avg TOV_y,Avg PF_y,Previous Game Data_y,Time between games_y,Win_pct_y
0,1148,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,"19,049",,1.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,40,94,.426,6,20,.300,13,18,.722,12,28,40,25,14,4,19,20,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,25,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,"16,312",,1.00,0,114.0,99.000000,44.000000,88.00,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2016-10-27,2 days,1.00,28,80,.350,5,18,.278,11,15,.733,4,35,39,16,9,5,20,20,0.00,1,97.000000,103.000000,36.0,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.00,10.000000,13.000000,22.000000,2016-10-26,3 days,0.000000
2,1176,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,"12,501",,1.00,1,109.0,85.500000,43.000000,86.50,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2016-10-29,2 days,1.00,36,75,.480,10,23,.435,13,19,.684,6,32,38,26,10,6,19,34,0.50,0,104.333333,99.666667,37.0,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.00,3.333333,13.666667,21.666667,2016-10-29,2 days,0.666667
3,1191,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,"13,800",,0.75,1,108.0,88.666667,39.666667,84.00,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2016-10-31,2 days,1.00,48,90,.533,9,22,.409,18,20,.900,11,31,42,24,11,9,15,25,0.40,0,103.250000,109.500000,38.0,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.75,3.750000,17.250000,24.000000,2016-11-01,1 days,0.250000
4,69,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,"14,663",,0.60,0,110.0,97.250000,40.000000,84.25,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2016-11-02,2 days,0.75,31,80,.388,5,20,.250,28,35,.800,11,35,46,13,7,3,19,23,0.25,1,101.666667,113.000000,40.0,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.00,4.333333,17.666667,21.000000,2016-11-02,2 days,0.000000


In [70]:
cleandNewDf5 = cleandNewDf4.drop(cleandNewDf4.loc[:,'FG_y':'PF_y'],axis = 1)

In [71]:
cleandNewDf = cleandNewDf5.drop(['index_x','overtimes_x','attendance_x'],axis =1)

In [72]:
cleandNewDf.head()

,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_team_pts_x,box_score_text_x,game_remarks_x,Team_win_pct_x,Home Team_x,Avg Pts_x,Avg Opp Pts_x,Avg FG_x,Avg FGA_x,Avg FG %_x,Avg 3P_x,Avg 3PA_x,Avg 3P%_x,Avg FT_x,Avg FTA_x,Avg FT%_x,Avg ORB_x,Avg DRB_x,Avg TRB_x,Avg AST_x,Avg STL_x,Avg BLK_x,Avg TOV_x,Avg PF_x,Previous Game Data_x,Time between games_x,Win_pct_x,Team_win_pct_y,Home Team_y,Avg Pts_y,Avg Opp Pts_y,Avg FG_y,Avg FGA_y,Avg FG %_y,Avg 3P_y,Avg 3PA_y,Avg 3P%_y,Avg FT_y,Avg FTA_y,Avg FT%_y,Avg ORB_y,Avg DRB_y,Avg TRB_y,Avg AST_y,Avg STL_y,Avg BLK_y,Avg TOV_y,Avg PF_y,Previous Game Data_y,Time between games_y,Win_pct_y
0,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,1.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,1.00,0,114.0,99.000000,44.000000,88.00,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2016-10-27,2 days,1.00,0.00,1,97.000000,103.000000,36.0,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.00,10.000000,13.000000,22.000000,2016-10-26,3 days,0.000000
2,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,1.00,1,109.0,85.500000,43.000000,86.50,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2016-10-29,2 days,1.00,0.50,0,104.333333,99.666667,37.0,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.00,3.333333,13.666667,21.666667,2016-10-29,2 days,0.666667
3,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,0.75,1,108.0,88.666667,39.666667,84.00,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2016-10-31,2 days,1.00,0.40,0,103.250000,109.500000,38.0,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.75,3.750000,17.250000,24.000000,2016-11-01,1 days,0.250000
4,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,0.60,0,110.0,97.250000,40.000000,84.25,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2016-11-02,2 days,0.75,0.25,1,101.666667,113.000000,40.0,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.00,4.333333,17.666667,21.000000,2016-11-02,2 days,0.000000


In [73]:
cleandNewDf = cleandNewDf.drop(['Team_win_pct_x','Team_win_pct_y'],axis =1)

In [74]:
cleandNewDf = cleandNewDf.drop(['Avg Opp Pts_x','Avg Opp Pts_y'],axis =1)

In [75]:
#Delete duplicates

dedupeData = cleandNewDf.drop_duplicates(subset ='box_score_text_x',keep = 'first')

dedupeData.head()

,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_team_pts_x,box_score_text_x,game_remarks_x,Home Team_x,Avg Pts_x,Avg FG_x,Avg FGA_x,Avg FG %_x,Avg 3P_x,Avg 3PA_x,Avg 3P%_x,Avg FT_x,Avg FTA_x,Avg FT%_x,Avg ORB_x,Avg DRB_x,Avg TRB_x,Avg AST_x,Avg STL_x,Avg BLK_x,Avg TOV_x,Avg PF_x,Previous Game Data_x,Time between games_x,Win_pct_x,Home Team_y,Avg Pts_y,Avg FG_y,Avg FGA_y,Avg FG %_y,Avg 3P_y,Avg 3PA_y,Avg 3P%_y,Avg FT_y,Avg FTA_y,Avg FT%_y,Avg ORB_y,Avg DRB_y,Avg TRB_y,Avg AST_y,Avg STL_y,Avg BLK_y,Avg TOV_y,Avg PF_y,Previous Game Data_y,Time between games_y,Win_pct_y
0,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,0,114.0,44.000000,88.00,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2016-10-27,2 days,1.00,1,97.000000,36.0,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.00,10.000000,13.000000,22.000000,2016-10-26,3 days,0.000000
2,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,1,109.0,43.000000,86.50,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2016-10-29,2 days,1.00,0,104.333333,37.0,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.00,3.333333,13.666667,21.666667,2016-10-29,2 days,0.666667
3,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,1,108.0,39.666667,84.00,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2016-10-31,2 days,1.00,0,103.250000,38.0,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.75,3.750000,17.250000,24.000000,2016-11-01,1 days,0.250000
4,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,0,110.0,40.000000,84.25,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2016-11-02,2 days,0.75,1,101.666667,40.0,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.00,4.333333,17.666667,21.000000,2016-11-02,2 days,0.000000


In [76]:
dedupeData.shape

(1135, 52)

In [77]:
dedupeData.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text',
       'game_remarks','Team Home?','Avg Team Pts', 
       'Avg Team FG', 'Avg Team FGA', 'Avg Team FG %', 'Avg Team 3P', 'Avg Team 3PA',
       'Avg Team 3P%', 'Avg Team FT', 'Avg Team FTA', 'Avg Team FT%', 'Avg Team ORB',
       'Avg Team DRB', 'Avg Team TRB', 'Avg Team AST', 'Avg Team STL', 'Avg Team BLK',
       'Avg Team TOV', 'Avg Team PF', 'Previous Game Data_x', 'Team Time between previous game',
       'Team_Win_Pct','Home Team_y', 'Avg Opp Pts',
       'Avg Opp FG', 'Avg Opp FGA', 'Avg Opp FG %', 'Avg Opp 3P', 'Avg Opp 3PA',
       'Avg Opp 3P%', 'Avg Opp FT', 'Avg Opp FTA', 'Avg Opp FT%', 'Avg Opp ORB',
       'Avg Opp DRB', 'Avg Opp TRB', 'Avg Opp AST', 'Avg Opp STL', 'Avg Opp BLK',
       'Avg Opp TOV', 'Avg Opp PF','Previous Game Data_y', 'Opp Time between previous game','Opp_Win_Pct']

In [78]:
dedupeCleanDf = dedupeData.drop(['Previous Game Data_x','Home Team_y','Previous Game Data_y'],axis = 1)

In [79]:
dedupeCleanDf.head()

,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,game_remarks,Team Home?,Avg Team Pts,Avg Team FG,Avg Team FGA,Avg Team FG %,Avg Team 3P,Avg Team 3PA,Avg Team 3P%,Avg Team FT,Avg Team FTA,Avg Team FT%,Avg Team ORB,Avg Team DRB,Avg Team TRB,Avg Team AST,Avg Team STL,Avg Team BLK,Avg Team TOV,Avg Team PF,Team Time between previous game,Team_Win_Pct,Avg Opp Pts,Avg Opp FG,Avg Opp FGA,Avg Opp FG %,Avg Opp 3P,Avg Opp 3PA,Avg Opp 3P%,Avg Opp FT,Avg Opp FTA,Avg Opp FT%,Avg Opp ORB,Avg Opp DRB,Avg Opp TRB,Avg Opp AST,Avg Opp STL,Avg Opp BLK,Avg Opp TOV,Avg Opp PF,Opp Time between previous game,Opp_Win_Pct
0,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,0,114.0,44.000000,88.00,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.0,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2 days,1.00,97.000000,36.0,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.00,10.000000,13.000000,22.000000,3 days,0.000000
2,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,1,109.0,43.000000,86.50,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.5,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2 days,1.00,104.333333,37.0,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.00,3.333333,13.666667,21.666667,2 days,0.666667
3,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,1,108.0,39.666667,84.00,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.0,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2 days,1.00,103.250000,38.0,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.75,3.750000,17.250000,24.000000,1 days,0.250000
4,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,0,110.0,40.000000,84.25,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.0,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2 days,0.75,101.666667,40.0,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.00,4.333333,17.666667,21.000000,2 days,0.000000


In [80]:
#Shift date to check if back-to-back game 

def tb2b(row):
    if pd.isnull(row['Team Time between previous game']):
        return(0)
    elif row['Team Time between previous game'].days == 1:
        return(1)
    return(0)

def ob2b(row):
    if pd.isnull(row['Opp Time between previous game']):
        return(0)
    elif row['Opp Time between previous game'].days == 1:
        return(1)
    return(0)
    
dedupeCleanDf['Team B2B?'] = dedupeCleanDf.apply(tb2b,axis = 1)
dedupeCleanDf['Opp B2B?'] = dedupeCleanDf.apply(ob2b,axis = 1)

In [81]:
dedupeCleanDf

,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,game_remarks,Team Home?,Avg Team Pts,Avg Team FG,Avg Team FGA,Avg Team FG %,Avg Team 3P,Avg Team 3PA,Avg Team 3P%,Avg Team FT,Avg Team FTA,Avg Team FT%,Avg Team ORB,Avg Team DRB,Avg Team TRB,Avg Team AST,Avg Team STL,Avg Team BLK,Avg Team TOV,Avg Team PF,Team Time between previous game,Team_Win_Pct,Avg Opp Pts,Avg Opp FG,Avg Opp FGA,Avg Opp FG %,Avg Opp 3P,Avg Opp 3PA,Avg Opp 3P%,Avg Opp FT,Avg Opp FTA,Avg Opp FT%,Avg Opp ORB,Avg Opp DRB,Avg Opp TRB,Avg Opp AST,Avg Opp STL,Avg Opp BLK,Avg Opp TOV,Avg Opp PF,Opp Time between previous game,Opp_Win_Pct,Team B2B?,Opp B2B?
0,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0,0
1,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,0,114.000000,44.000000,88.000000,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.000000,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2 days,1.000000,97.000000,36.000000,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.000000,10.000000,13.000000,22.000000,3 days,0.000000,0,0
2,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,1,109.000000,43.000000,86.500000,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.500000,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2 days,1.000000,104.333333,37.000000,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.000000,3.333333,13.666667,21.666667,2 days,0.666667,0,0
3,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,1,108.000000,39.666667,84.000000,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.000000,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2 days,1.000000,103.250000,38.000000,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.750000,3.750000,17.250000,24.000000,1 days,0.250000,0,1
4,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,0,110.000000,40.000000,84.250000,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.000000,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2 days,0.750000,101.666667,40.000000,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.000000,4.333333,17.666667,21.000000,2 days,0.000000,0,0
5,2016-11-05,7:30p,Atlanta Hawks,112,Houston Rockets,97,https://www.basketball-reference.com/boxscores...,,1,106.400000,38.400000,84.000000,0.456000,8.800000,26.600000,0.324600,20.800000,30.600000,0.695800,11.400000,33.800000,45.200000,27.000000,11.400000,4.800000,16.400000,19.200000,1 days,0.600000,110.200000,39.200000,83.200000,0.470600,13.000000,35.200000,0.365200,18.800000,26.200000,0.712200,10.800000,31.800000,42.600000,24.200000,7.600000,3.200000,15.000000,22.400000,3 days,0.600000,1,0
6,2016-11-08,7:00p,Atlanta Hawks,110,Cleveland Cavaliers,106,https://www.basketball-reference.com/boxscores...,,0,107.333333,39.500000,84.166667,0.468167,8.666667,25.500000,0.337167,19.666667,29.500000,0.677000,11.666667,33.333333,45.000000,27.333333,12.333333,5.000000,16.833333,18.166667,3 days,0.666667,112.333333,39.166667,86.500000,0.453167,13.333333,34.666667,0.387167,20.666667,26.166667,0.779833,10.500000,34.166667,44.666667,22.833333,8.000000,4.500000,12.500000,19.833333,3 days,1.000000,0,0
7,2016-11-09,7:30p,Atlanta Hawks,115,Chicago Bulls,107,https://

In [82]:
#East Team, West Team

nbaDictwest = {
'Atlanta Hawks': 0,
'Brooklyn Nets': 0,
'Boston Celtics': 0,
'Charlotte Hornets': 0,
'Chicago Bulls': 0,
'Cleveland Cavaliers': 0,
'Dallas Mavericks': 1,
'Denver Nuggets': 1,
'Detroit Pistons': 0,
'Golden State Warriors': 1,
'Houston Rockets': 1,
'Indiana Pacers': 0,
'Los Angeles Clippers': 1,
'Los Angeles Lakers': 1,
'Memphis Grizzlies': 1,
'Miami Heat': 0,
'Milwaukee Bucks': 0,
'Minnesota Timberwolves': 1,
'New Orleans Pelicans': 1,
'New York Knicks': 0,
'Oklahoma City Thunder': 1,
'Orlando Magic': 0,
'Philadelphia 76ers': 0,
'Phoenix Suns': 1,
'Portland Trail Blazers': 1,
'Sacramento Kings': 1,
'San Antonio Spurs': 1,
'Toronto Raptors': 0,
'Utah Jazz': 0,
'Washington Wizards': 0}

dedupeCleanDf['Team West?'] = dedupeCleanDf.apply(lambda row: nbaDictwest[row['team_name']],axis=1)
dedupeCleanDf['Opp West?'] = dedupeCleanDf.apply(lambda row: nbaDictwest[row['opposing_team']],axis=1)

In [83]:
dedupeCleanDf

,date_game,game_start_time,team_name,team_pts,opposing_team,opposing_team_pts,box_score_text,game_remarks,Team Home?,Avg Team Pts,Avg Team FG,Avg Team FGA,Avg Team FG %,Avg Team 3P,Avg Team 3PA,Avg Team 3P%,Avg Team FT,Avg Team FTA,Avg Team FT%,Avg Team ORB,Avg Team DRB,Avg Team TRB,Avg Team AST,Avg Team STL,Avg Team BLK,Avg Team TOV,Avg Team PF,Team Time between previous game,Team_Win_Pct,Avg Opp Pts,Avg Opp FG,Avg Opp FGA,Avg Opp FG %,Avg Opp 3P,Avg Opp 3PA,Avg Opp 3P%,Avg Opp FT,Avg Opp FTA,Avg Opp FT%,Avg Opp ORB,Avg Opp DRB,Avg Opp TRB,Avg Opp AST,Avg Opp STL,Avg Opp BLK,Avg Opp TOV,Avg Opp PF,Opp Time between previous game,Opp_Win_Pct,Team B2B?,Opp B2B?,Team West?,Opp West?
0,2016-10-27,7:30p,Atlanta Hawks,114,Washington Wizards,99,https://www.basketball-reference.com/boxscores...,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0,0,0,0
1,2016-10-29,12:30p,Atlanta Hawks,104,Philadelphia 76ers,72,https://www.basketball-reference.com/boxscores...,,0,114.000000,44.000000,88.000000,0.500000,12.000000,26.000000,0.462000,14.000000,18.000000,0.778000,14.000000,38.000000,52.000000,28.000000,13.000000,7.000000,21.000000,19.000000,2 days,1.000000,97.000000,36.000000,84.000000,0.429000,7.000000,24.000000,0.292000,18.000000,25.000000,0.720000,5.000000,38.000000,43.000000,21.000000,5.000000,10.000000,13.000000,22.000000,3 days,0.000000,0,0,0,0
2,2016-10-31,7:30p,Atlanta Hawks,106,Sacramento Kings,95,https://www.basketball-reference.com/boxscores...,,1,109.000000,43.000000,86.500000,0.497000,8.500000,23.000000,0.356000,14.500000,20.500000,0.715000,10.000000,39.500000,49.500000,29.500000,12.000000,7.000000,16.500000,16.500000,2 days,1.000000,104.333333,37.000000,80.333333,0.457000,7.000000,22.333333,0.310667,23.333333,29.666667,0.777667,9.000000,28.333333,37.333333,23.333333,6.000000,3.333333,13.666667,21.666667,2 days,0.666667,0,0,0,1
3,2016-11-02,7:30p,Atlanta Hawks,116,Los Angeles Lakers,123,https://www.basketball-reference.com/boxscores...,,1,108.000000,39.666667,84.000000,0.470667,9.333333,25.333333,0.359667,19.333333,29.333333,0.682333,12.333333,35.000000,47.333333,27.666667,11.666667,6.666667,15.333333,17.666667,2 days,1.000000,103.250000,38.000000,87.250000,0.434500,8.500000,29.500000,0.291750,18.750000,23.000000,0.813000,11.000000,30.750000,41.750000,17.750000,9.750000,3.750000,17.250000,24.000000,1 days,0.250000,0,1,0,1
4,2016-11-04,7:00p,Atlanta Hawks,92,Washington Wizards,95,https://www.basketball-reference.com/boxscores...,,0,110.000000,40.000000,84.250000,0.473500,9.750000,27.000000,0.355750,20.250000,29.250000,0.710000,11.750000,33.000000,44.750000,28.250000,11.250000,5.250000,16.000000,18.000000,2 days,0.750000,101.666667,40.000000,84.000000,0.483667,4.666667,18.333333,0.266667,17.000000,22.333333,0.756333,8.666667,30.000000,38.666667,21.666667,10.000000,4.333333,17.666667,21.000000,2 days,0.000000,0,0,0,0
5,2016-11-05,7:30p,Atlanta Hawks,112,Houston Rockets,97,https://www.basketball-reference.com/boxscores...,,1,106.400000,38.400000,84.000000,0.456000,8.800000,26.600000,0.324600,20.800000,30.600000,0.695800,11.400000,33.800000,45.200000,27.000000,11.400000,4.800000,16.400000,19.200000,1 days,0.600000,110.200000,39.200000,83.200000,0.470600,13.000000,35.200000,0.365200,18.800000,26.200000,0.712200,10.800000,31.800000,42.600000,24.200000,7.600000,3.200000,15.000000,22.400000,3 days,0.600000,1,0,0,1
6,2016-11-08,7:00p,Atlanta Hawks,110,Cleveland Cavaliers,106,https://www.basketball-reference.com/boxscores...,,0,107.333333,39.500000,84.166667,0.468167,8.666667,25.500000,0.337167,19.666667,29.500000,0.677000,11.666667,33.333333,45.000000,27.333333,12.333333,5.000000,16.833333,18.166667,3 days,0.666667,112.333333,39.166667,86.500000,0.453167,13.333333,34.666667,0.387167,20.666667,26.166667,0.779833,10.500000,34.166667,44.666667,22.833333,8.000000,4.500000,12.500000,19.833333,3 days,1.000000,0,0,0,0
7,2016-11-09,7

In [84]:
#Output to csv for saving purposes
export_csv = dedupeCleanDf.to_csv (r'C:\Users\jeromerufin\Desktop\Metis\nba_16_17.csv') #Don't forget to add '.csv' at the end of the path
